In [23]:
import os

from langchain_community.retrievers import (
    AzureAISearchRetriever,
)

from config_notebooks import *
from langchain.agents import AgentType, initialize_agent
from pprint import pprint

from langchain.chains import RetrievalQA
from langchain_core.tools import Tool
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain.chat_models import AzureChatOpenAI

In [24]:
action_input = "Compare green house gas emissions in Chap02,Chap09"
action_input = "Compare Scope 3 approach in Chap02,Chap09"

In [25]:
os.environ["AZURE_AI_SEARCH_SERVICE_NAME"] = AZURE_SEARCH_SERVICE_ENDPOINT
os.environ["AZURE_AI_SEARCH_INDEX_NAME"] = AZURE_SEARCH_INDEX_NAME
os.environ["AZURE_AI_SEARCH_API_KEY"] = AZURE_SEARCH_ADMIN_KEY

In [26]:
retriever_02 = AzureAISearchRetriever(
    content_key="content", top_k=3, index_name=AZURE_SEARCH_INDEX_NAME,
    filter="sourcepage eq 'AGL.pdf'"
)
retriever_09 = AzureAISearchRetriever(
    content_key="content", top_k=3, index_name=AZURE_SEARCH_INDEX_NAME,
    filter="sourcepage eq 'coca-cola-business-sustainability-report-2022.pdf'"
)

In [27]:
os.environ["AZURE_OPENAI_API_KEY"] =AZURE_OPENAI_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] =AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_VERSION"] = "2023-05-15"


In [28]:
llm = AzureChatOpenAI(azure_deployment=AZURE_OPENAI_DEPLOYMENT_GPT_4O_ID,temperature=0)

In [29]:
class DocumentInput(BaseModel):
    question: str = Field()

In [30]:
FILE_NAME_02 = "CHAP02"
FILE_NAME_09 = "CHAP09"

In [31]:
tool_02 = Tool(
    args_schema=DocumentInput,
            name=FILE_NAME_02,
            description=f"useful when you want to answer questions about {FILE_NAME_02}",
            func=RetrievalQA.from_chain_type(llm=llm, retriever=retriever_02),
        )
tool_09 = Tool(
    args_schema=DocumentInput,
            name=FILE_NAME_09,
            description=f"useful when you want to answer questions about {FILE_NAME_09}",
            func=RetrievalQA.from_chain_type(llm=llm, retriever=retriever_09),
        )
tools = [tool_02, tool_09]

In [32]:
agent = initialize_agent(
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=tools,
    llm=llm,
    verbose=True,
)

results = agent({"input": action_input})



> Entering new AgentExecutor chain...

Invoking: `CHAP02` with `{'question': 'What is the Scope 3 approach?'}`


{'query': 'What is the Scope 3 approach?', 'result': "The Scope 3 approach refers to a category within the Greenhouse Gas (GHG) Protocol that encompasses all indirect emissions that occur in a company's value chain, excluding those covered in Scope 2. Scope 3 emissions include all sources not within an organization’s direct control but related to its activities, such as emissions from purchased goods and services, business travel, employee commuting, waste disposal, and the use of sold products. This approach is crucial for companies aiming to understand and reduce their overall carbon footprint, as it often represents the largest portion of their total emissions."}
Invoking: `CHAP09` with `{'question': 'What is the Scope 3 approach?'}`


{'query': 'What is the Scope 3 approach?', 'result': "The Scope 3 approach refers to a category within the Greenhouse Gas (GHG) Protocol

In [33]:
pprint(results["output"])

('The Scope 3 approach in both Chap02 and Chap09 refers to the category within '
 'the Greenhouse Gas (GHG) Protocol that encompasses all indirect emissions '
 "occurring in a company's value chain. However, there are some differences in "
 'the details provided by each chapter:\n'
 '\n'
 '### Chap02:\n'
 '- **Definition**: Scope 3 includes all indirect emissions not covered in '
 "Scope 2, related to a company's activities.\n"
 '- **Examples**: Emissions from purchased goods and services, business '
 'travel, employee commuting, waste disposal, and the use of sold products.\n'
 '- **Focus**: Emphasizes the importance of understanding and reducing the '
 'overall carbon footprint, as Scope 3 often represents the largest portion of '
 'total emissions.\n'
 '\n'
 '### Chap09:\n'
 '- **Definition**: Similar to Chap02, it includes all indirect emissions in a '
 "company's value chain.\n"
 '- **Categorization**: Divides Scope 3 emissions into upstream and downstream '
 'activities.\n'
 '  -